# Assignment 1.2: Word2vec preprocessing (20 points)

Preprocessing is not the most exciting part of NLP, but it is still one of the most important ones. Your task is to preprocess raw text (you can use your own, or [this one](http://mattmahoney.net/dc/text8.zip). For this task text preprocessing mostly consists of:

1. cleaning (mostly, if your dataset is from social media or parsed from the internet)
1. tokenization
1. building the vocabulary and choosing its size. Use only high-frequency words, change all other words to UNK or handle it in your own manner. You can use `collections.Counter` for that.
1. assigning each token a number (numericalization). In other words, make word2index и index2word objects.
1. data structuring and batching - make X and y matrices generator for word2vec (explained in more details below)

**ATTN!:** If you use your own data, please, attach a download link. 

Your goal is to make **Batcher** class which returns two numpy tensors with word indices. It should be possible to use one for word2vec training. You can implement batcher for Skip-Gram or CBOW architecture, the picture below can be helpful to remember the difference.

![text](https://raw.githubusercontent.com/deepmipt/deep-nlp-seminars/651804899d05b96fc72b9474404fab330365ca09/seminar_02/pics/architecture.png)

There are several ways to do it right. Shapes could be `x_batch.shape = (batch_size, 2*window_size)`, `y_batch.shape = (batch_size,)` for CBOW or `(batch_size,)`, `(batch_size, 2*window_size)` for Skip-Gram. You should **not** do negative sampling here.

They should be adequately parametrized: CBOW(window_size, ...), SkipGram(window_size, ...). You should implement only one batcher in this task; and it's up to you which one to chose.

Useful links:
1. [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
1. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
1. [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

You can write the code in this notebook, or in a separate file. It can be reused for the next task. The result of your work should represent that your batch has a proper structure (right shapes) and content (words should be from one context, not some random indices). To show that, translate indices back to words and print them to show something like this:

```
text = ['first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']

window_size = 2

# CBOW:
indices_to_words(x_batch) = \
        [['first', 'used', 'early', 'working'],
        ['used', 'against', 'working', 'class'],
        ['against', 'early', 'class', 'radicals'],
        ['early', 'working', 'radicals', 'including']]

indices_to_words(labels_batch) = ['against', 'early', 'working', 'class']
```

In [1]:
import numpy as np
import torch
from torch.utils import data
from collections import Counter
import re

In [2]:
class CorpusDataset_CBOW(data.Dataset):
    def __init__(self, data_link, window_size=4):
        # text preprocessing
        words_tokenized = self.read_preprocess_text(data_link)
        # creating training samples: ([context_array, target_word])
        self.context_target =  [([words_tokenized[i-(j+1)] for j in range(window_size)] + [words_tokenized[i+(j+1)] for j in range(window_size)],
                                words_tokenized[i]) for i in range(window_size, len(words_tokenized)-window_size)]
        # vocabulary of words
        self.vocab = Counter(words_tokenized)
        # word2index mapping
        self.word2index = {word_tuple[0]: idx for idx, word_tuple in enumerate(self.vocab.most_common())}
        # index2word mapping
        self.index2word = list(self.word2index.keys())
        self.vocab_size = len(self.vocab)
        self.window_size = window_size
        
    def read_preprocess_text(self, data_link):
        text = open(data_link, 'r').readline()
        text = re.sub(r"[^a-zA-Z0-9]+", ' ', text)
        text = text.split(' ')
        counter_words = Counter(text)
        res = []
        for word in text:
            if counter_words[word] > 5:
                res.append(word)
            else:
                res.append('UNK')
        return np.array(res)

    def __getitem__(self, idx):
        context = torch.tensor([self.word2index[w] for w in self.context_target[idx][0]])
        context_words = self.context_target[idx][0]
        target = torch.tensor([self.word2index[self.context_target[idx][1]]])
        target_words = self.context_target[idx][1]
        return context, target, context_words, target_words

    def __len__(self):
        return len(self.context_target)

In [3]:
text_data = CorpusDataset_CBOW('text8', window_size = 4)
dataloader = data.DataLoader(text_data, batch_size = 1)

In [4]:
context, target, words_c, words_t = next(iter(dataloader))

In [5]:
print(context, words_c)

tensor([[  12, 3081, 5234,    6,  195,    1, 3134,   46]]) [('as',), ('originated',), ('anarchism',), ('UNK',), ('term',), ('of',), ('abuse',), ('first',)]


In [6]:
target, words_t

(tensor([[5]]), ('a',))